# 一、构建训练文本

In [14]:
import os
import json
import pandas as pd

In [15]:
# 配置
INPUT_CSV  = "../../map_with_asr_split.csv" 
OUTPUT_CSV = "../../train_master.csv"

# 要拼接的 JSON 字段（按重要性排序）
JSON_KEYS = [
    "title",          # 商品标题
    "商品名称",        # 同 title，可能略有差异
    "label",          # 细分类：背包
    "Llabel",         # 大分类：出行用品
    "商品分类",        # 同 Llabel
    "商品价格",        # 价格
    "店铺名称",        # 店铺
    "所属地区名称",    # 地区
]

SEP = " | "

In [16]:
def load_json(json_path):
    """加载单条 JSON 记录"""
    try:
        with open(json_path, "r", encoding="utf-8") as f:
            return json.load(f)
    except: 
        return {}

In [12]:
def build_text(row):
    parts = []
    
    # 解析 JSON 指定字段
    data = load_json(row["json_path"])
    for key in JSON_KEYS:
        val = data.get(key)
        if val and str(val).strip():
            parts.append(f"{key}:{str(val).strip()}")
    
    # 追加 ASR 文本
    asr = row.get("asr_text", "")
    if isinstance(asr, str) and asr.strip():
        parts.append(f"ASR:{asr.strip()}")
    
    return SEP.join(parts)

In [ ]:
df = pd.read_csv(INPUT_CSV)
df["text"] = df. apply(build_text, axis=1)
df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")

print(f"Done.  {len(df)} rows → {OUTPUT_CSV}")
print(f"\n字段统计：")
print(f"  - 总条数: {len(df)}")
print(f"  - text 非空:  {df['text']. notna().sum()}")

Done.  1000 rows → ../../train_master.csv

字段统计：
  - 总条数: 1000
  - text 非空:  1000

示例 text (第1条)：
title:GYXX/光影行星宙斯Z双肩包男背包机能旅行大容量机车骑行电脑书包 | 商品名称:GYXX/光影行星宙斯Z双肩包男背包机能旅行大容量机车骑行电脑书包 | label:背包 | Llabel:出行用品 | 商品分类:出行用品 | 商品价格:585 | 店铺名称:光影行星旗舰店 | 所属地区名称:广东 | ASR:GYXX光影行星周四次一双舰包南背包技能旅行大容量汽车齐行电脑收包
